In [9]:
# https://github.com/pixegami/rag-tutorial-v2
import os
import shutil
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from IPython.display import Markdown
import textwrap
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.bedrock import BedrockEmbeddings

In [3]:

def load_documents(DATA_PATH):
    document_loader = PyPDFLoader(DATA_PATH)
    return document_loader.load()

In [4]:
DATA_PATH = 'data/AE-260225-140604.pdf'
docs = load_documents(DATA_PATH)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107910e50>>
Traceback (most recent call last):
  File "/Users/martin.rojo/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [8]:

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


In [9]:
spl_doc = split_documents(docs)

In [13]:
# Supongamos que 'docs' es una lista de objetos Document
spl_doc = [doc for doc in spl_doc if int(doc.metadata.get('page', 0)) >= 5]


In [15]:
spl_doc

[Document(metadata={'source': 'data/AE-260225-140604.pdf', 'page': 5, 'page_label': '6'}, page_content='6\nAnalytics Engineering\n\xa0\n\xa0\n  \n   ¿Quiénes somos?\nAgustina Leiva                 Analytics Engineering Manager\n\xa0\n\xa0\nLourdes Almada        BI Analyst\n\xa0\nGabriela G. Wallis Data Engineer\n\xa0\nErika Rojas               Data Engineer\n\xa0\n  Rubén Jara        Data Engineer\n\xa0\n\xa0\nLucas Trubiano      Data Engineer\n\xa0\n\xa0\nLuciana Correa      Data Engineer'),
 Document(metadata={'source': 'data/AE-260225-140604.pdf', 'page': 6, 'page_label': '7'}, page_content='7\n\xa0\nMatias Tomasone       BI Analyst\n      Luis Bobadilla         Data Engineer\n\xa0\n     Nahuel Carro          Data Engineer\n\xa0\n\xa0\n\xa0\nMarcos Audisio    Data Engineer\n\xa0\n\xa0\nMauricio Fara      Data Engineer\n\xa0\n\xa0\n     \n   Canales de comunicación \n#analytics-engineering Privado Canal de comunicación interna del equipo \nAnalytics Engineering.\n#analytics-engineeri

In [11]:
model_id = "amazon.titan-embed-text-v1"
region_name="us-east-1"


def get_embedding_function():
    embeddings = BedrockEmbeddings(
        region_name=region_name,
        model_id=model_id,
    )
    return embeddings


In [12]:
get_embedding_function().embed_query('hola')

[0.984375,
 -0.451171875,
 0.33203125,
 0.427734375,
 0.69140625,
 1.0625,
 0.28515625,
 -0.00130462646484375,
 -0.373046875,
 -0.048583984375,
 0.357421875,
 0.79296875,
 0.625,
 -0.024658203125,
 -0.2109375,
 -0.26953125,
 0.2734375,
 0.71484375,
 -1.078125,
 1.078125,
 -0.01544189453125,
 0.703125,
 0.1240234375,
 0.279296875,
 -0.45703125,
 -0.2578125,
 0.2119140625,
 -0.95703125,
 0.94140625,
 -1.28125,
 0.44140625,
 0.373046875,
 -0.625,
 0.00653076171875,
 -0.53515625,
 -0.400390625,
 0.08935546875,
 -0.95703125,
 0.2236328125,
 0.08740234375,
 -0.5625,
 -1.0234375,
 1.25,
 1.109375,
 1.1953125,
 -0.50390625,
 0.189453125,
 -0.62890625,
 -0.60546875,
 1.2734375,
 0.146484375,
 0.98828125,
 -0.2431640625,
 -0.064453125,
 -0.1845703125,
 -0.1376953125,
 0.236328125,
 -0.76171875,
 0.890625,
 -0.671875,
 -1.0,
 -0.357421875,
 -0.234375,
 -0.8828125,
 0.59765625,
 -0.240234375,
 -0.890625,
 0.056640625,
 0.15625,
 0.67578125,
 -0.400390625,
 -0.330078125,
 0.05908203125,
 0.62109375

In [23]:
def calculate_chunk_ids(chunks):

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return chunks

In [ ]:
CHROMA_PATH = 'chroma_db'

def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    chunks_with_ids = calculate_chunk_ids(chunks)

    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

In [ ]:
CHROMA_PATH = 'chroma_db'
add_to_chroma(spl_doc)

Number of existing documents in DB: 0
👉 Adding new documents: 1991


/var/folders/n0/hftbhvf94sj1gzrc6vglbzyw0000gp/T/ipykernel_74436/3005455283.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [14]:
CHROMA_PATH = 'chroma_db'

db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

In [15]:
db.similarity_search_with_score('que es el data lake?', k=5)

[(Document(metadata={'id': 'data/AE-260225-140604.pdf:247:1', 'page': 247, 'page_label': '248', 'source': 'data/AE-260225-140604.pdf'}, page_content='1. ¿Qué es un Data Lake?\nUn Data Lake es un sistema de almacenamiento centralizado que permite guardar grandes volúmenes de datos en su formato original, ya sean estructurados, semiestructurados o no estructurados. Su propósito es conservar los datos en bruto para analizarlos o procesarlos según sea necesario.'),
  109.49423217773438),
 (Document(metadata={'id': 'data/AE-260225-140604.pdf:247:0', 'page': 247, 'page_label': '248', 'source': 'data/AE-260225-140604.pdf'}, page_content='248\nData Lake\n1. ¿Qué es un Data Lake?2. Estructura de nuestro Data Lake en AWS2.0. Fuentes de Datos2.1. Ingesta2.2. Datos Crudos2.3. Desarrollo2.4. Modelado de Datos2.5. Métricas, Lógica de Negocio y KPIs2.6. Exportación de Datos3. ¿Cómo consultar el Data Lake?\n1. ¿Qué es un Data Lake?'),
  142.25242614746094),
 (Document(metadata={'id': 'data/AE-260225-1

In [16]:
from langchain_aws import ChatBedrock

txt_model_name = "amazon.nova-lite-v1:0"

def get_llm():
    return ChatBedrock(
        region_name=region_name,
        model_id=txt_model_name,
        model_kwargs={
            "temperature": 0.2,
            "topP": 0.8,
            "maxTokenCount": 2048,

        }, 
    )


In [18]:
from langchain_aws import ChatBedrock

chat = ChatBedrock(
        region_name='us-east-1',
        model_id=txt_model_name,
        model_kwargs={
            "temperature": 0.2,
            "topP": 0.5,
            "maxTokenCount": 400,

        }, 
    )

# Realizar la predicción
respuesta = chat.predict('mensajes')

/var/folders/n0/hftbhvf94sj1gzrc6vglbzyw0000gp/T/ipykernel_94588/3019596216.py:15: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = chat.predict('mensajes')


In [19]:
respuesta

'¡Hola! Parece que estás interesado en enviar o recibir mensajes. Aquí tienes algunas ideas sobre diferentes tipos de mensajes que podrías querer compartir:\n\n### Mensajes Personales\n- **A un amigo:**\n  "Hola [Nombre], ¿cómo estás? He estado pensando en ti y me gustaría que pasáramos un rato juntos pronto. ¿Qué te parece si nos vemos este fin de semana?"\n\n- **A un ser querido:**\n  "Querido [Nombre], quiero que sepas cuánto significas para mí. Eres una parte importante de mi vida y aprecio cada momento que pasamos juntos. Te quiero mucho."\n\n### Mensajes Profesionales\n- **A un cliente:**\n  "Estimado [Nombre], gracias por su interés en nuestros productos/servicios. Estamos aquí para ayudarle y estamos disponibles para responder cualquier pregunta que tenga. ¡Esperamos poder trabajar juntos!"\n\n- **A un compañero de trabajo:**\n  "Hola [Nombre], quería agradecerte por tu ayuda con el proyecto [Nombre del Proyecto]. Tu contribución fue invaluable y aprecio mucho tu colaboración."

In [85]:
from langchain_aws import BedrockLLM
from langchain.schema import AIMessage, HumanMessage

# Define your AWS region - replace with your actual region
region_name = "us-east-1"
model_id = "amazon.nova-lite-v1:0"

def get_bedrock_llm():
    """
    Function to create a properly configured BedrockLLM instance for the Nova Lite model
    """
    llm = BedrockLLM(
        model_id=model_id,
        region_name=region_name,
        model_kwargs={
            "temperature": 0.2,
            "top_p": 0.8,
            "max_tokens": 2048,
            # Required for Nova Lite chat models
            "messages": []  # This will be populated with actual messages when we invoke
        }
    )
    return llm

In [77]:
import json

request_body = {
    "messages": [
        {
            "role": "user",
            "content": "¿Cuál es la capital de España?"
        }
    ],
    "temperature": 0.2,
    "top_p": 0.8,
    "max_tokens": 2048
}

request_json = json.dumps(request_body)


In [ ]:
llm = get_llm()
llm.generate([{"messages":"hola"}])

ERROR:root:Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [messages] not found, please reformat your input and try again.


ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [messages] not found, please reformat your input and try again.

In [ ]:
from utils import generar_prompt, get_chroma

generar_prompt(chunks="si", question="¿Qué es un data lake?")

ImportError: cannot import name 'get_chroma' from 'utils' (/Users/martin.rojo/Desktop/ueno/onboarding/utils.py)

In [2]:
db = get_chroma()

In [2]:
from utils import inference

inference("¿Qué es un data lake?")

'Un Data Lake es un sistema de almacenamiento centralizado diseñado para almacenar grandes volúmenes de datos en su formato original, sin necesidad de transformaciones previas. Estos datos pueden ser estructurados, semiestructurados o no estructurados, lo que permite una flexibilidad y capacidad de almacenamiento sin precedentes.\n\n### Características Principales de un Data Lake:\n\n1. **Variedad de Formatos**: Un Data Lake puede almacenar diferentes tipos de datos, incluyendo archivos de texto, JSON, CSV, XML, imágenes, videos, y otros formatos no estructurados.\n2. **Escalabilidad**: Puede crecer sin límites, tanto en términos de almacenamiento como de usuarios y aplicaciones que acceden a los datos.\n3. **Conservación de Datos en Bruto**: Los datos se almacenan tal como son recibidos, sin modificaciones, lo que permite mantener la integridad y autenticidad de los datos originales.\n4. **Acceso Flexible**: Permite el acceso y análisis de datos utilizando una variedad de herramientas

In [ ]:
from utils import get_chroma, generar_prompt, obtener_respuesta
db = get_chroma()
res = db.similarity_search('que es el data lake?', k=3)
context = "\n\n---\n\n".join([doc.page_content for doc in res])

generar_prompt(chunks=context, question="¿Qué es un data lake?")

obtener_respuesta(generar_prompt(chunks=context, question="¿Qué es un data lake?"))


'\n        Eres un asistente virtual especializado en Analytics Engineering. \n        Se te proporcionarán fragmentos de información relevantes.\n        Tu objetivo es utilizar estos fragmentos para responder de manera precisa y detallada a las consultas del equipo.\n        Mantén un tono profesional y orientado a soluciones en todas tus respuestas.\n\n        Conexto:\n        1. ¿Qué es un Data Lake?\nUn Data Lake es un sistema de almacenamiento centralizado que permite guardar grandes volúmenes de datos en su formato original, ya sean estructurados, semiestructurados o no estructurados. Su propósito es conservar los datos en bruto para analizarlos o procesarlos según sea necesario.\n\n---\n\n248\nData Lake\n1. ¿Qué es un Data Lake?2. Estructura de nuestro Data Lake en AWS2.0. Fuentes de Datos2.1. Ingesta2.2. Datos Crudos2.3. Desarrollo2.4. Modelado de Datos2.5. Métricas, Lógica de Negocio y KPIs2.6. Exportación de Datos3. ¿Cómo consultar el Data Lake?\n1. ¿Qué es un Data Lake?\n\

In [4]:
obtener_respuesta(generar_prompt(chunks=context, question="¿Qué es un data lake?"))

'Un Data Lake es un sistema de almacenamiento centralizado diseñado para guardar grandes volúmenes de datos en su formato original, sin importar si estos son estructurados, semiestructurados o no estructurados. Su propósito principal es conservar los datos en bruto, permitiendo su análisis o procesamiento según las necesidades específicas.\n\nA continuación, se detallan algunos aspectos clave de un Data Lake:\n\n1. **Versatilidad en el Almacenamiento**: Un Data Lake puede almacenar una amplia variedad de tipos de datos, incluyendo archivos de texto, documentos, imágenes, videos, registros de transacciones, y mucho más. Esto permite una gestión integral de la información.\n\n2. **Escalabilidad**: Los Data Lakes están diseñados para crecer con las necesidades de la organización, pudiendo manejar petabytes de datos sin problemas.\n\n3. **Acceso Flexible**: Los datos se pueden acceder y analizar utilizando una variedad de herramientas y tecnologías, lo que facilita la integración con difer